# LAB 10 

In [3]:
import mysql.connector

# Database credentials
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Duke#7539',
    'database': 'lab10'
}

# Establishing connection to MySQL
conn = mysql.connector.connect(**db_config)
if conn.is_connected():
    print('Connected to MySQL database!')
    cursor = conn.cursor()
    # Execute SQL query
    cursor.execute("SELECT VERSION()")
    data = cursor.fetchone()
    print("Database version:", data)
    # Close cursor and connection
    cursor.close()
    conn.close()
    print('Connection closed.')


Connected to MySQL database!
Database version: ('8.1.0',)
Connection closed.


In [8]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = 'world_development_data.csv'  # Replace 'your_file_path.csv' with your CSV file path
df = pd.read_csv(file_path)

# Display all columns and rows
pd.set_option('display.max_columns', None)  # To display all columns
pd.set_option('display.max_rows', None)     # To display all rows
print(df.head())


   Year           Country  Region           SubRegion   IntermRegion  \
0  1973       Afghanistan    Asia       Southern Asia            NaN   
1  1973       Netherlands  Europe      Western Europe            NaN   
2  1973            Poland  Europe      Eastern Europe            NaN   
3  1973  Egypt, Arab Rep.  Africa     Northern Africa            NaN   
4  1973             Gabon  Africa  Sub-Saharan Africa  Middle Africa   

   SurfAreaSqKm     PopDens  PopGrowth%           GDP  GDPGrowth%  \
0      652860.0   17.747275    2.524421  1.733333e+09         NaN   
1       41530.0  398.084182    0.827331  7.184091e+10    5.444223   
2      312690.0  108.907245    0.908114           NaN         NaN   
3     1001450.0   37.290448    2.151183  1.009853e+10    3.506830   
4      267670.0    2.465510    2.067382  7.227807e+08   10.182434   

   AdolFertRate  AgriValAdd%GDP  DomCredit%GDP  Exports%GDP  FertRate  \
0       142.874             NaN            NaN    12.948718     7.487   
1     

Table 1

In [9]:
import pandas as pd
import mysql.connector

# Database credentials
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Duke#7539',
    'database': 'lab10'
}

# Initial CountryID value
country_id = 100

# Establishing connection to MySQL
conn = mysql.connector.connect(**db_config)

if conn.is_connected():
    cursor = conn.cursor()

    # Read CSV file into a pandas DataFrame
    file_path = 'world_development_data.csv'  # Replace with your CSV file path
    data = pd.read_csv(file_path)

    # Mapping between CSV column names and table column names
    column_mapping = {
        'Country': 'CountryName',
        'Region': 'Region',
        'SubRegion': 'SubRegion',
        'IntermRegion': 'IntermRegion',
        'SurfAreaSqKm': 'SurfAreaSqKm',
        'PopDens': 'PopDens',
        'PopGrowth%': 'PopGrowth'
    }

    # Map columns according to the provided pattern
    data = data.rename(columns=column_mapping)

    # Insert data into the CountryInfo table with assigned CountryID values
    for _, row in data.iterrows():
        # Check for NaN (null) values and replace them with None
        row = row.where(pd.notnull(row), None)

        insert_query = """
            INSERT INTO CountryInfo (CountryID, CountryName, Region, SubRegion, IntermRegion, SurfAreaSqKm, PopDens, PopGrowth)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """
        values = (
            country_id,
            row['CountryName'],
            row['Region'],
            row['SubRegion'],
            row['IntermRegion'],
            row['SurfAreaSqKm'],
            row['PopDens'],
            row['PopGrowth']
        )
        cursor.execute(insert_query, values)

        # Increment CountryID for the next row insertion
        country_id += 1

    # Commit changes and close connection
    conn.commit()
    cursor.close()
    conn.close()
    print('Data inserted successfully.')
else:
    print('Connection failed.')


Data inserted successfully.


Table 2

In [43]:
import pandas as pd

# Read the CSV file
file_path = 'world_development_data.csv'  # Replace with your CSV file path
data = pd.read_csv(file_path)

# Columns in the CSV file
csv_columns = data.columns.tolist()

# Column names for Economic Indicators table
table_columns = [
    'Year', 'CountryID', 'GDP', 'GDPGrowth', 'AdolFertRate', 'AgriValAddGDP', 'DomCreditGDP', 'ExportsGDP',
    'FertRate', 'FDINetBoP', 'GNICapAtlas', 'GNIAtlas', 'GrossCapFormGDP', 'ImportsGDP', 'IndValAddGDP'
]

# Check how many columns from the CSV match the table columns
matching_columns = set(csv_columns) & set(table_columns)
num_matching_columns = len(matching_columns)

print(f"Number of matching columns between CSV and table: {num_matching_columns}")
print(f"Matching columns: {matching_columns}")


Number of matching columns between CSV and table: 6
Matching columns: {'GDP', 'FertRate', 'AdolFertRate', 'Year', 'GNIAtlas', 'FDINetBoP'}


In [15]:
import pymysql
import pandas as pd
from fuzzywuzzy import process

# Read the CSV file
file_path = 'world_development_data.csv'  # Replace with your CSV file path
data = pd.read_csv(file_path)

# Specified column names for Economic Indicators table
table_columns = [
    'Year', 'GDP', 'GDPGrowth', 'AdolFertRate', 'AgriValAddGDP', 'DomCreditGDP', 'ExportsGDP',
    'FertRate', 'FDINetBoP', 'GNICapAtlas', 'GNIAtlas', 'GrossCapFormGDP', 'ImportsGDP', 'IndValAddGDP'
]

# Columns in the CSV file
csv_columns = data.columns.tolist()

# Find similar column names using fuzzy matching
similar_columns = {}
for table_column in table_columns:
    matches = process.extract(table_column, csv_columns, limit=1)
    if matches[0][1] > 80:  # Adjust the threshold as needed (80 is an example threshold)
        similar_columns[table_column] = matches[0][0]

# Database connection configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Duke#7539',
    'database': 'lab10'
}

# Connect to the database
connection = pymysql.connect(**db_config)
cursor = connection.cursor()

# Insert values into the Economic Indicators table
for index, row in data.iterrows():
    country_id = 100 + index  # Starting CountryID at 100 and auto-incrementing
    values = [row[similar_columns[col]] if col in similar_columns and pd.notnull(row[similar_columns[col]]) else None
              for col in table_columns]
    placeholders = ', '.join(['%s'] * len(table_columns))
    query = f"INSERT INTO EconomicIndicators (CountryID, {', '.join(table_columns)}) VALUES " \
            f"({country_id}, {placeholders})"
    cursor.execute(query, values)
    connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()


DataError: (1264, "Out of range value for column 'FDINetBoP' at row 1")

In [1]:
import pymysql

# Database connection configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Duke#7539'  # Replace with your MySQL root password
}

# Database name to be dropped
database_name = 'lab10'

try:
    # Connect to MySQL
    connection = pymysql.connect(**db_config)
    cursor = connection.cursor()

    # SQL command to drop the database
    drop_database_query = f"DROP DATABASE {database_name}"

    # Execute the query to drop the database
    cursor.execute(drop_database_query)
    print(f"Database '{database_name}' dropped successfully.")

    # Close the cursor and connection
    cursor.close()
    connection.close()

except pymysql.Error as e:
    print(f"Error: {e}")


Error: (1008, "Can't drop database 'lab10'; database doesn't exist")
